# <center>Analyzing NYC Traffic Collision Data</center>
## <center>A 15-688 Project</center><br/><center>*by*<br/> Ahmet Emre Unal (ahmetemu)<br/>*and*<br/>Marco Peyrot (mpeyrotc)</center>

In [ ]:
print 'Hello, world!'